# Basic Tutorial on using the dMRI Microstructure Toolbox

Work in Progress...  
Author: Dr. Rutger Fick  
Date : 23-10-2017

In this tutorial we will show the basic usage of the dMRI Microstructure Toolbox.

To generate and fit dMRI data originating from a single voxel:

- Some basics about dMRI acquisition parameters, and what information is needed to use the toolbox.
- How to load, set model parameters, and generate diffusion signals, and fit data using individual biophysical models.
- How to combine biophysical models into a combined "microstructure model".
- How to impose parameter linking between different components in a microstructure model, such as a Tortuosity constraint.
- How to generate and fit data with microstructure models.

Finally, to fit N-dimensional data sets with any model we also explain how to set voxel-dependent initial conditions for the fitting process.

# Loading acquisition parameters and making an acquisition scheme

The first thing to do is to load the parameters of some acquisition scheme. Typically, the b-values and gradient directions, along with the used pulse duration time $\delta$ and pulse separation time $\Delta$ are known. The Microstruktur toolbox uses SI units, so be careful, as bvalues are typically saved in $s/mm^2$, not $s/m^2$.

As an example we load the acquisition parameters of the WU-MINN Human Connectome Project:

In [8]:
# load the necessary modules
from microstruktur.signal_models import three_dimensional_models
from microstruktur.acquisition_scheme.acquisition_scheme import acquisition_scheme_from_bvalues
from os.path import join
import numpy as np

# the HCP acquisition parameters are saved in the following toolbox path:
acquisition_path = three_dimensional_models.GRADIENT_TABLES_PATH

# we can then load the parameters themselves and convert them to SI units:
bvalues = np.loadtxt(join(acquisition_path, 'bvals_hcp_wu_minn.txt'))  # given in s/mm^2
bvalues = bvalues * 1e6  # now given in SI units as s/m^2
gradient_directions = np.loadtxt(join(acquisition_path, 'bvecs_hcp_wu_minn.txt'))  # on the unit sphere

# The delta and Delta times we know from the HCP documentation in seconds
delta = 0.0106  
Delta = 0.0431 

# The acquisition scheme we use in the toolbox is then simply created as follows:
acq_scheme = acquisition_scheme_from_bvalues(bvalues, gradient_directions, delta, Delta)

Once the acquisition scheme object is made, we also have access to all metrics that can be derived from the given input parameters. Values such as qvalues, diffusion times, gradient strengths, and to which shell a DWI belongs can be obtained with the following commands:

In [12]:
acq_scheme.gradient_strengths;  # the gradient strength in T/m
acq_scheme.qvalues;  # describes the diffusion sensitization in 1/m
acq_scheme.tau;  # diffusion time as Delta - delta / 3. in seconds
acq_scheme.shell_indices;  # integer unique to every shell. 0 is automically assigned to b0 measurements

# Simulating and fitting data using a cylinder stick model

To get started, we will start with 

In [16]:
# simple stick example generating data and fitting
stick = three_dimensional_models.I1Stick()
gt_mu = np.random.rand(2)
gt_lambda_par = (np.random.rand() + 1.) * 1e-9
gt_parameter_vector = stick.parameters_to_parameter_vector(
    lambda_par=gt_lambda_par, mu=gt_mu)

E = stick.simulate_signal(acq_scheme, gt_parameter_vector)

x0 = stick.parameters_to_parameter_vector(
    lambda_par=(np.random.rand() + 1.) * 1e-9,
    mu=np.random.rand(2)
)
res = stick.fit(E, acq_scheme, x0)

In [17]:
# making simple stick and ball generating data and fitting
stick = three_dimensional_models.I1Stick()
ball = three_dimensional_models.E3Ball()

ball_and_stick = (
    three_dimensional_models.PartialVolumeCombinedMicrostrukturModel(
        models=[ball, stick],
        parameter_links=[],
        optimise_partial_volumes=True)
)
gt_mu = np.clip(np.random.rand(2), .3, np.inf)
gt_lambda_par = (np.random.rand() + 1.) * 1e-9
gt_lambda_iso = gt_lambda_par / 2.
gt_partial_volume = 0.3

gt_parameter_vector = ball_and_stick.parameters_to_parameter_vector(
    I1Stick_1_lambda_par=gt_lambda_par,
    E3Ball_1_lambda_iso=gt_lambda_iso,
    I1Stick_1_mu=gt_mu,
    partial_volume_0=gt_partial_volume
)

E = ball_and_stick.simulate_signal(
    acq_scheme, gt_parameter_vector)

x0 = ball_and_stick.parameters_to_parameter_vector(
    I1Stick_1_lambda_par=(np.random.rand() + 1.) * 1e-9,
    E3Ball_1_lambda_iso=gt_lambda_par / 2.,
    I1Stick_1_mu=np.random.rand(2),
    partial_volume_0=np.random.rand()
)
res = ball_and_stick.fit(E, acq_scheme, x0)

In [19]:
# simple stick and zeppelin example with parameter linking
from microstruktur.signal_models.utils import (
    T1_tortuosity, parameter_equality)

gt_mu = np.clip(np.random.rand(2), .3, np.inf)
gt_lambda_par = (np.random.rand() + 1.) * 1e-9
gt_partial_volume = 0.3

stick = three_dimensional_models.I1Stick()
zeppelin = three_dimensional_models.E4Zeppelin()

parameter_links_stick_and_tortuous_zeppelin = [
    (  # tortuosity assumption
        zeppelin, 'lambda_perp',
        T1_tortuosity, [
            (None, 'partial_volume_0'),
            (stick, 'lambda_par')
        ]
    ),
    (  # equal parallel diffusivities
        zeppelin, 'lambda_par',
        parameter_equality, [
            (stick, 'lambda_par')
        ]
    ),
    (  # equal parallel diffusivities
        zeppelin, 'mu',
        parameter_equality, [
            (stick, 'mu')
        ]
    )
]

stick_and_tortuous_zeppelin = (
    three_dimensional_models.PartialVolumeCombinedMicrostrukturModel(
        models=[stick, zeppelin],
        parameter_links=parameter_links_stick_and_tortuous_zeppelin,
        optimise_partial_volumes=True)
)

gt_parameter_vector = (
    stick_and_tortuous_zeppelin.parameters_to_parameter_vector(
        I1Stick_1_lambda_par=gt_lambda_par,
        I1Stick_1_mu=gt_mu,
        partial_volume_0=gt_partial_volume)
)

E = stick_and_tortuous_zeppelin.simulate_signal(
    acq_scheme, gt_parameter_vector)

In [22]:
# multi-dimensional example with voxel-dependent initial condition
stick = three_dimensional_models.I1Stick()
ball = three_dimensional_models.E3Ball()
ball_and_stick = (
    three_dimensional_models.PartialVolumeCombinedMicrostrukturModel(
        models=[ball, stick],
        parameter_links=[],
        optimise_partial_volumes=True)
)
gt_lambda_par = (np.random.rand() + 1.) * 1e-9
gt_lambda_iso = gt_lambda_par / 2.
gt_partial_volume = 0.3
gt_mu_array = np.empty((10, 10, 2))

# I'm putting the orientation of the stick all over the sphere.
for i, mu1 in enumerate(np.linspace(0, np.pi, 10)):
    for j, mu2 in enumerate(np.linspace(-np.pi, np.pi, 10)):
        gt_mu_array[i, j] = np.r_[mu1, mu2]

gt_parameter_vector = (
    ball_and_stick.parameters_to_parameter_vector(
        I1Stick_1_lambda_par=gt_lambda_par,
        E3Ball_1_lambda_iso=gt_lambda_iso,
        I1Stick_1_mu=gt_mu_array,
        partial_volume_0=gt_partial_volume)
)

E_array = ball_and_stick.simulate_signal(
    acq_scheme, gt_parameter_vector)

# I'm giving a voxel-dependent initial condition with gt_mu_array
res = ball_and_stick.fit(E_array, acq_scheme,
                         gt_parameter_vector)